## Variation of origin-test-reader

In [1]:
#!/usr/bin/env python

import sys
import os
import random
import time
import zmq
import json
import numpy
import matplotlib.pyplot as plt
import math

# first find ourself
fullBinPath  = os.path.abspath(os.getcwd() + "/" + sys.argv[0])
fullBasePath = os.path.dirname(os.path.dirname(fullBinPath))
fullLibPath  = os.path.join(fullBasePath, "lib")
fullLibPath='C:\Users\Rb\LabSoftware\Origin\lib'
fullCfgPath  = os.path.join(fullBasePath, "config")
fullCfgPath='C:\Users\Rb\LabSoftware\Origin\config'
sys.path.append(fullLibPath)
print fullLibPath
from origin.client import server, random_data

if len(sys.argv) > 1:
  if sys.argv[1] == 'test':
    configfile = os.path.join(fullCfgPath, "origin-server-test.cfg")
  else:
    configfile = os.path.join(fullCfgPath, sys.argv[1])
else:
  configfile = os.path.join(fullCfgPath, "origin-server.cfg")

import ConfigParser
config = ConfigParser.ConfigParser()
config.read(configfile)

context = zmq.Context()
socket = context.socket(zmq.REQ)
#host = config.get('Server','ip')
host='hexlabmonitor.physics.wisc.edu'

#port = config.getint('Server','read_port')
port=5561
socket.connect("tcp://%s:%s" % (host,port))

#stream_test_list = ['ULETrans960']
stream_test_list = ['Rb_CoilThermistor']

C:\Users\Rb\LabSoftware\Origin\lib


In [2]:
def RtoT(R):
    # Coefficients
    [A,B,C]=[0.001125308852122, 0.000234711863267,0.000000085663516]
    if R>0:
        oneoverT=A+B*math.log(R)+C*math.pow(math.log(R),3)
        T=1.0/oneoverT-273.15
        return T
    else:
        return None

def dataconditioning(x):
    if x<10 and x>0:
        return x*10000.0
    elif x>0:
        return x
    else:
        return None

def to32bit(timestamp):
    return float(timestamp)/2**32

In [ ]:
starttime=time.time()-60*60*24
stoptime=time.time()-10 # now
for stream in stream_test_list:
    print "sending raw read request for stream `{}`....".format(stream)
    request_obj = { 'start':starttime, 'stop':stoptime,'stream': stream, 'raw': True }
    socket.send(json.dumps(request_obj))
    response = socket.recv()
    print "Data received"        

sending raw read request for stream `Rb_CoilThermistor`....


In [ ]:
dict=json.loads(response)
indicator=dict[0]
truedata=dict[1]

In [ ]:
fig=plt.figure(figsize=(10,10))
Tdata=numpy.array(map(RtoT,map(dataconditioning,truedata['thermistor'])))
timedata=(numpy.array(map(to32bit,truedata['measurement_time']))-time.time())/3600
plt.plot(timedata,Tdata)
plt.axis([min(timedata),max(timedata),numpy.median(Tdata)-2,numpy.median(Tdata)+2])
plt.ylabel('3DMOT Z Coil temperature(C)')
plt.xlabel('Hours from current time')
plt.show()